# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [67]:
#include <stdio.h>

int main(){
    float s = 200, n = 5, j = 1;
    int i;
    for (i = 0; i<n; i++){
        printf("%.2f",s);
        s *= (1+j/100);
        if (i!=4)
            printf("; ");
    }
    return 0;
}

200.00; 202.00; 204.02; 206.06; 208.12

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [68]:
#include <stdio.h>

float proximaParcela(float s, float j){
    return s*(1+j/100);
}

int main(){
    float s = 200, n = 5, j = 1;
    int i;
    for (i = 0; i<n; i++){
        printf("%.2f",s);
        s = proximaParcela(s, j);
        if (i!=4)
            printf("; ");
    }
    return 0;
}

200.00; 202.00; 204.02; 206.06; 208.12

## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [69]:
#include <stdio.h>

typedef struct{
    float s, n, j, parc;
} emprestimo;

float proximaParcela(emprestimo emp){
    return emp.parc*(1+emp.j/100);
}

int main(){
    emprestimo dados;
    dados.parc = dados.s = 200;
    dados.n = 5;
    dados.j = 1;
    int i;
    for (i = 0; i<dados.n; i++){
        printf("%.2f",dados.parc);
        dados.parc = proximaParcela(dados);
        if (i!=4)
            printf("; ");
    }
    return 0;
}

200.00; 202.00; 204.02; 206.06; 208.12

## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [73]:
#include <stdio.h>
#include <stdlib.h>

typedef struct{
    float n, j, *parc;
    int i;
} emprestimo;

emprestimo novo(int s, int n, int j){
    emprestimo novo;
    novo.parc = malloc(novo.n*sizeof(float));
    novo.parc[0] = s;
    novo.n = n;
    novo.j = j;
    novo.i = 0;
    return novo;
}

void pagarparc(emprestimo emp){
    emp.parc[emp.i+1] = emp.parc[emp.i]*(1+emp.j/100);      
    printf("%.2f", emp.parc[emp.i]);
}

int main(){
    int i, n = 5;
    emprestimo emp = novo(200, n, 1);
    for (i = 0; i<n; i++){
        pagarparc(emp);
        emp.i++;
        if (i!=4)
            printf("; ");
    }
    free(emp.parc);
    return 0;
}

200.00; 202.00; 204.02; 206.06; 208.12

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [30]:
#include <stdio.h>
#include <stdlib.h>

typedef struct{
    float j, *parc;
    int i, n, flag;
} emprestimo;

emprestimo novo(int s, int n, int j){
    emprestimo novo;
    novo.parc = malloc(novo.n*sizeof(float));
    novo.parc[0] = s;
    novo.n = n;
    novo.j = j;
    novo.i = novo.flag = 0;
    return novo;
}

void pagarparc(emprestimo emp, int j){
    if (emp.i < emp.n-1)
        emp.parc[emp.i+1] = emp.parc[emp.i]*(1+emp.j/100);
    printf("Emprestimo %d: parcela %d eh %.2f\n", j+1, emp.i+1, emp.parc[emp.i]);
}

int main(){
    int i, j, n, m = 0, k = 0;
    float s, juros;
    emprestimo caderno[100];
    for (i = 0; i<100; i++){
        //scanf("%f %d %f", &s, &n, &juros); //<- linha adequada para o caso genérico, substituida pelas 9 pŕoximas linhas para executar o exemplo dado
        if (i==0){
            s = 200; n = 5; juros = 1;
        }
        if (i==1){
            s = 500; n = 7; juros = 2;
        }
        if (i>1){
            n = 0;
        }
        if (n!= 0){             // nenhum empréstimo tem 0 parcelas, boa sugestão para indicar "EOF"
            caderno[i] = novo(s, n, juros);
            m++;                // após iteração, "m", armazenará quantos empréstimos foram recebidos
        }
        if (k == 0 || n > k)    // e "k" armazenará o maior numero de parcelas recebido
            k = n;
    }
    for (i = 0; i<k; i++){
        for (j = 0; j<m; j++){
            if (!caderno[j].flag){     // "se empréstimo ainda não foi completamente pago, então..."
                pagarparc(caderno[j], j);
                caderno[j].i++;
                if (i==caderno[j].n-1)
                    caderno[j].flag = 1;
            }
        }
    }
    for (i = 0; i<m; i++)
        free(caderno[i].parc);
    return 0;
}

corrupted size vs. prev_size


Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08


[C kernel] Executable exited with code -6